# AutoMPC Demo

Welcome!  This notebook demonstrates the core features of AutoMPC.  We will use the Cart-Pole swing-up task as an example, and we will tune an MPC pipeline consisting of a multi-layer perception (MLP) system ID model and an iLQR optimizer.

## Set-Up

To begin, we need an input dataset and specifications for the system and task.  The `benchmarks` module provides these for a few example systems.

In [ ]:
import autompc as ampc
import numpy as np
from autompc.benchmarks import CartpoleSwingupBenchmark

benchmark = CartpoleSwingupBenchmark()


# Get system and task specification
system = benchmark.system
task   = benchmark.task

# Generate benchmark dataset
trajs = benchmark.gen_trajs(seed=100, n_trajs=500, traj_len=200)

Next, we need to declare our MPC pipeline.  The following code initializes a pipeline with a MLP system ID model, a quadratic objective function, and and an iLQR optimizer.

In [ ]:
from autompc.sysid import MLPFactory
from autompc.control import IterativeLQRFactory
from autompc.costs import QuadCostFactory

model_factory = MLPFactory(system)
ctrlr_factory = IterativeLQRFactory(system)
cost_factory  = QuadCostFactory(system)

pipeline = ampc.Pipeline(system, model_factory, ctrlr_factory, cost_factory)

We can view the joint configuration space of the pipeline

In [ ]:
pipeline.get_configuration_space()

## Tuning

Tune with selected surrogate model class, default model configuration.

In [ ]:
from autompc.tuning import PipelineTuner

surrogate_factory = MLPFactory(system)
tuner = PipelineTuner(surrogate_mode="defaultcfg", surrogate_factory=surrogate_factory, surrogate_split=0.5)

controller, tune_result = tuner.run(pipeline, task, trajs, n_iters=100, rng=np.random.default_rng(100))

We can also set the configuration of the surrogate model.

In [ ]:
surrogate_cfg = surrogate_factory.get_configuration_space().get_default_configuration()
surrogate_cfg["n_hidden_layers"] = "2"
surrogate_cfg["hidden_size_1"] = 128
surrogate_cfg["hidden_size_2"] = 128       
surrogate_cfg["nonlintype"] = "relu"

tuner = PipelineTuner(surrogate_mode="fixedcfg", surrogate_factory=surrogate_factory,
                     surrogate_cfg=surrogate_cfg, surrogate_split=0.5)

controller, tune_result2 = tuner.run(pipeline, task, trajs, n_iters=100, rng=np.random.default_rng(100),
                                    truedyn=benchmark.dynamics)

We can also auto-tune a particular surrogate class

In [ ]:
tuner = PipelineTuner(surrogate_mode="autotune", surrogate_factory=surrogate_factory, surrogate_split=0.5)

controller, tune_result3 = tuner.run(pipeline, task, trajs, n_iters=100, rng=np.random.default_rng(100),
                                    truedyn=benchmark.dynamics)

Finally, we can auto-select and auto-tune the surrogate mode class

In [ ]:
tuner = PipelineTuner(surrogate_mode="autoselect", surrogate_split=0.5)

controller, tune_result4 = tuner.run(pipeline, task, trajs, n_iters=100, rng=np.random.default_rng(100),
                                    truedyn=benchmark.dynamics)

Finally, we can pre-train and select our own surrogate.

In [ ]:
from autompc.tuning import ModelTuner
from autompc.evaluation import HoldoutModelEvaluator

sysid_trajs = trajs[:250]
surr_trajs = trajs[250:]

model_evaluator = HoldoutModelEvaluator(system=system,trajs=surr_trajs,rng=np.random.default_rng(100),
                                        holdout_prop=0.25, metric="rmse")
model_tuner = ModelTuner(system, model_evaluator)
model_tuner.add_model_factory(surrogate_factory)

surrogate_model, _ = model_tuner.run(rng=np.random.default_rng(100), n_iters = 1)

tuner = PipelineTuner(surrogate_mode="pretrain")

controller, tune_result5 = tuner.run(pipeline, task, sysid_trajs, n_iters=100, surrogate=surrogate_model,
                                    truedyn=benchmark.dynamics)

## Decoupled Tuning

The above examples is full pipeline tuning, which searches the configuration space of all components simultaneously.  Alternatively, we can take a decoupled tuning approach, where the model is first tuned based on prediction accuracy, then the objective function and optimizer are tuned.

First, we must tune the model

In [ ]:
from autompc.tuning import ModelTuner
from autompc.evaluation import HoldoutModelEvaluator

model_evaluator = HoldoutModelEvaluator(holdout_prop=0.25, metric="rmse", trajs=trajs, system=system,
                                       rng=np.random.default_rng(100))
model_tuner = ModelTuner(system, model_evaluator)
model_tuner.add_model_factory(model_factory)

model, model_tune_result = model_tuner.run(rng=np.random.default_rng(100), n_iters=2)

Now that we have tuned the model, we can create a pipeline with the pre-tuned model, and then run pipeline tuning.

In [ ]:
pipeline_fixed_model = ampc.Pipeline(system, model, iLQRFactory, QuadCostFactory)
controller6, tune_result6 = tuner.run(pipeline, task, trajs, n_iters=100, truedyn=benchmark.dynamics)   